# PINN Testing

In [ ]:
import os
from dotenv import load_dotenv
import sys

In [ ]:
load_dotenv()

lib_dir = os.environ.get("LIB_DIR")
sys.path.append(lib_dir)

data_dir = os.environ.get("DATA_DIR")
model_dir = os.path.join(data_dir, "models")
points_file_name = "points_final.csv"

In [ ]:
import numpy as np
import pandas as pd
import torch
import yaml
import plotly.express as px
import plotly.graph_objects as go
from pinn import PINN

In [ ]:
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
wing_df = pd.read_csv(os.path.join(data_dir, points_file_name))

device = torch.device("cpu")

x_min, y_min, z_min = wing_df.min()
x_max, y_max, z_max = wing_df.max()

overall_min = min(x_min, y_min, z_min)
overall_max = max(x_max, y_max, z_max)

xyz_range = (overall_min, overall_max)

In [ ]:
def show_wing():
  df = px.data.iris()
  fig = px.scatter_3d(wing_df, x='x', y='y', z='z')
  fig.update_traces(marker_size = 1)
  fig.update_traces(marker_color = 'slategrey')
  fig.update_layout(
      scene = dict(
          xaxis = dict(range=xyz_range),
          yaxis = dict(range=xyz_range),
          zaxis = dict(range=xyz_range)))

  fig.show()

show_wing()

In [ ]:
# load model
model_name = config["model_name"]
model_checkpoint_num = config["model_checkpoint_num"]

pinn = PINN.load_from_checkpoint_for_testing(model_dir, model_name, model_checkpoint_num).to(device)

In [ ]:
# testing domain
x_max = config["x_max"]
y_max = config["y_max"]
z_max = config["z_max"]
# t_max = 2 # TODO: set t>2 to test outside of training domain

Nx = config["Nx"]
Ny = config["Ny"]
Nz = config["Nz"]
# Nt = config["Nt"]

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
# dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
# t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_grid = np.meshgrid(x_test, y_test, z_test, indexing='ij')

# testing points
xyz_combinations = torch.cartesian_prod(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32), torch.tensor(z_test, dtype=torch.float32))
num_points = xyz_combinations.shape[0]

# predict
input_xyz = xyz_combinations.to(device)
output_uvwp = pinn(input_xyz)

u_test = output_uvwp[:, 0].to("cpu").detach().numpy()
v_test = output_uvwp[:, 1].to("cpu").detach().numpy()
w_test = output_uvwp[:, 2].to("cpu").detach().numpy()
p_test = output_uvwp[:, 3].to("cpu").detach().numpy()
velocity_norm = np.linalg.norm([u_test, v_test, w_test], axis=0)

# min & max values of the model outputs

velocity_norm = np.array(velocity_norm)
min_velocity_norm = velocity_norm.min()
max_velocity_norm = velocity_norm.max()

min_pressure = np.array(p_test).min()
max_pressure = np.array(p_test).max()

In [ ]:
_scatter = go.Scatter3d(
    x=wing_df['x'],
    y=wing_df['y'],
    z=wing_df['z'],
    mode='markers',
    marker=dict(size=1, color='slategrey')
)

_cone = go.Cone(
    x=x_grid.flatten(),
    y=y_grid.flatten(),
    z=z_grid.flatten(),
    u=u_test,
    v=v_test,
    w=w_test,
    colorscale='jet',
    cmin=min_velocity_norm,
    cmax=max_velocity_norm,
    colorbar_title='Velocity<br>Magnitude',
    opacity=0.1,
    sizemode="scaled",
    sizeref=max_velocity_norm/6
)

# Create figure
fig = go.Figure(data=[_scatter, _cone])

fig.update_layout(
        scene = dict(
          xaxis=dict(range=[0, x_max], autorange=False),
          yaxis=dict(range=[0, y_max], autorange=False),
          zaxis=dict(range=[0, z_max], autorange=False),
        ))

fig.show()

In [ ]:
_scatter = go.Scatter3d(
    x=wing_df['x'],
    y=wing_df['y'],
    z=wing_df['z'],
    mode='markers',
    marker=dict(size=1, color='slategrey')
)

_scatter_p = go.Scatter3d(
    x=x_grid.flatten(),
    y=y_grid.flatten(),
    z=z_grid.flatten(),
    mode='markers',
    marker=dict(
      size=4,  # Adjust the size of the marker if needed
      color=p_test,  # Set the color of the markers based on the pressure values
      colorscale='jet',  # Choose a colorscale (e.g., Viridis, Jet, etc.)
      cmin=min_pressure,
      cmax=max_pressure,
      colorbar=dict(title='Pressure'),  # Optional colorbar to indicate the scale
      opacity=0.1  # Adjust the opacity if needed
    )
)

# Create figure
fig = go.Figure(data=[_scatter, _scatter_p])

fig.update_layout(
        scene = dict(
          xaxis=dict(range=[0, x_max], autorange=False),
          yaxis=dict(range=[0, y_max], autorange=False),
          zaxis=dict(range=[0, z_max], autorange=False),
        ))

fig.show()

In [ ]:
# slice domain
_y=.7

xyz_arr = np.array([[_x, _y, _z] for _x in x_test for _z in z_test])
input_xyz = torch.tensor(xyz_arr, dtype=torch.float32)

output_uvwp = pinn(input_xyz)

u_test = output_uvwp[:, 0].to("cpu").detach().numpy()
v_test = output_uvwp[:, 1].to("cpu").detach().numpy()
w_test = output_uvwp[:, 2].to("cpu").detach().numpy()
p_test = output_uvwp[:, 3].to("cpu").detach().numpy()
velocity_norm = np.linalg.norm([u_test, v_test, w_test], axis=0)

In [ ]:
fig = go.Figure(data =
    go.Contour(
        z=velocity_norm.reshape((Nx, Nz)),
        x=x_test,
        y=z_test,
        colorscale='jet',
        colorbar=dict(title='Velocity Norm'),
        ncontours=150,
        contours_showlines=False,
        transpose=True,
        zmin=min_velocity_norm,
        zmax=max_velocity_norm,
    ))

fig.update_layout(
        scene = dict(
          xaxis=dict(range=[0, x_max], autorange=False),
          yaxis=dict(range=[0, z_max], autorange=False)
        ))

fig.show()

In [ ]:
fig = go.Figure(data =
    go.Contour(
        z=p_test.reshape((Nx, Nz)),
        x=x_test,
        y=z_test,
        colorscale='jet',
        colorbar=dict(title='Pressure'),
        ncontours=150,
        contours_showlines=False,
        transpose=True,
        zmin=min_pressure,
        zmax=max_pressure
    ))

fig.update_layout(
        scene = dict(
          xaxis=dict(range=[0, x_max], autorange=False),
          yaxis=dict(range=[0, z_max], autorange=False)
        ))

fig.show()